In [5]:
import os
import numpy as np
import pandas as pd
# path='/usr/people/out/Documents/380+220kV_extended'

In [6]:
path='/usr/people/out/Documents/Finland' #aanpassen
netwerk=pd.read_csv(path+'/spreadsheetcables.csv', delimiter = ';')

# begin loop to write values
coord1=pd.DataFrame(columns=['lon', 'lat'])
coord2=pd.DataFrame(columns=['lon', 'lat'])
coord4=pd.DataFrame(columns=['glon', 'glat', 'nr'])

for line in range(len(netwerk)): # put end locations under start location for gmt, so that you double the lines
    coord1.at[2*line,'lon']=netwerk.at[line,'strtlon'] 
    coord1.at[2*line,'lat']=netwerk.at[line,'strtlat'] 
    coord1.at[2*line+1,'lon']=netwerk.at[line,'eindlon']
    coord1.at[2*line+1,'lat']=netwerk.at[line,'eindlat'] 
    
    coord4.at[line,'glon']=(netwerk.at[line,'strtlon']+netwerk.at[line,'eindlon'])/2. #get numbers and mean location cables
    coord4.at[line,'glat']=(netwerk.at[line,'strtlat']+netwerk.at[line,'eindlat'])/2.
    coord4.at[line,'nr']=str(netwerk.at[line,'kabelnr'])
    
    # if statement to spot discontinuities
    if line>0 and coord1.at[2*line-1,'lat']!=coord1.at[2*line,'lat'] and coord1.at[2*line-1,'lon']!=coord1.at[2*line,'lon']:
#         print(f'line= {line}')
        coord3=pd.DataFrame([[coord1.at[2*line,'lon'], coord1.at[2*line,'lat']]], columns=['lon', 'lat']) #create new dataframe
        coord1.at[2*line,'lon']='>'+str(coord1.at[2*line,'lon']) #add > for gmt
        coord2=coord2.append(coord1.loc[2*line]) #append it
        coord2=coord2.append(coord3) #append old one, otherwise no line will be drawn
        coord2=coord2.append(coord1.loc[2*line+1]) #append the one after
        del coord3
    else:
        coord2=coord2.append(coord1.loc[2*line])
        coord2=coord2.append(coord1.loc[2*line+1])

#write to a file with no header and column titles
coord2.to_csv(path_or_buf=path+'/cables.csv', sep=' ', index=False, header=False)
coord4.to_csv(path_or_buf=path+'/cables+naam.csv', sep=' ', index=False, header=False)

In [8]:
# now extract trafo data
netwerk=pd.read_csv(path+'/spreadsheettrafo.csv', delimiter = ';')
trafo1=pd.DataFrame(columns=['lon', 'lat'])
trafo2=pd.DataFrame(columns=['lon', 'lat', 'naam'])
for line in range(len(netwerk)): #alleen locatie
    trafo1.at[line,'lon']=netwerk.at[line,'lon']
    trafo1.at[line,'lat']=netwerk.at[line,'lat']
    
for line in range(len(netwerk)): #met naam
    trafo2.at[line,'lon']=netwerk.at[line,'lon']
    trafo2.at[line,'lat']=netwerk.at[line,'lat']
    trafo2.at[line,'nr']=str(netwerk.at[line,'trafonr'])

    
trafo1.to_csv(path_or_buf=path+'/trafo.csv', sep=' ', index=False, header=False)
trafo2.to_csv(path_or_buf=path+'/trafo+naam.csv', sep=' ', index=False, header=False)

In [9]:
# create output for Northern 220V electrical system

proj='-JM16C -P'
lims='-R18/32/59.5/70'
psfile=f'{path}/Finland.ps' #aanpassen
os.system(f'gmt pscoast {proj} {lims} -W0.5p -K -Ggrey -Slightblue -Df -N1/0.25p> {psfile}' )
os.system(f'gmt psbasemap {proj} {lims} -Ba1g1 -BWeSn+t"High-voltage network Netherlands" -O -K>> {psfile}' )
# os.system(f'gmt psxy {path}/cables220.csv {proj} {lims} -W0.5p -Wblue -O -K>> {psfile}' )
# os.system(f'gmt psxy {path}/cables380.csv {proj} {lims} -W0.5p -Wred -O -K>> {psfile}' )
os.system(f'gmt psxy {path}/cables.csv {proj} {lims} -W0.5p -Wred -O -K>> {psfile}' )
# os.system(f'gmt pstext {path}/cables+naam.csv {proj} {lims} -F+f8p,Helvetica,black -O -K >> {psfile}')
os.system(f'gmt psxy {path}/trafo.csv {proj} {lims} -Ss0.2c -Gblack -O -K>> {psfile}')
os.system(f'gmt pstext {path}/trafo+naam.csv {proj} {lims} -F+f8pHelvetica,black -O>> {psfile}')

0

In [2]:
# overzichtskaart stations and area voor SECS
path='/usr/people/out/Documents/overview_map'
proj='-JM15C -P'
lims='-R-14/21/42/65'
psfile=f'{path}/map_overview.ps'
os.system(f'gmt pscoast {proj} {lims} -W0.25p -Ggrey -Slightblue -N1/0.25p -K> {psfile}' )
os.system(f'gmt psbasemap {proj} {lims} -Ba5 -BWeSn -O -K>> {psfile}' )
os.system(f'gmt psxy {path}/current_area {proj} {lims} -Sc0.1cm -Gdarkgreen -O -K>> {psfile}')
os.system(f'gmt psxy {path}/station_coordinates {proj} {lims} -Sc0.2i -Gred -O -K>> {psfile}')
os.system(f'gmt pstext {path}/station_coordinatestext {proj} {lims} -F+f22pHelvetica,black -O -K>> {psfile}')
os.system(f'gmt psxy {path}/Dutch_area {proj} {lims} -W2p -Wblack -O -K>> {psfile}' )
os.system(f'gmt psxy {path}/legend {proj} {lims} -Sc0.2i -Gdarkgreen -O -K>> {psfile}')
os.system(f'gmt psxy {path}/legend2 {proj} {lims} -Sc0.2i -Gred -O -K>> {psfile}')
os.system(f'gmt psxy {path}/legend3 {proj} {lims} -Ss0.2i -Wblack -W2p -O -K>> {psfile}')
os.system(f'gmt pstext {path}/legendtext {proj} {lims} -F+f20pHelvetica,black -O -K>> {psfile}')
os.system(f'gmt pstext {path}/legendtext2 {proj} {lims} -F+f20pHelvetica,black -O -K>> {psfile}')
os.system(f'gmt pstext {path}/legendtext3 {proj} {lims} -F+f20pHelvetica,black -O>> {psfile}')
os.system(f'convert -density 300 {psfile} {path}/map_overview.png')

0

In [1]:
from multiprocessing import Process
import os
from multiprocessing import Lock
import logging

lock=Lock()
lock2=Lock()

# magnetische contouren latitude
path='/usr/people/out/Documents/Magnetic_field/station_results'
day='large_29-10-2003' #real map location
day2="29-10-2003" #what the name should be

logging.basicConfig(filename=f'{path}/{day}/logbookplot.log', level=logging.DEBUG, format='%(asctime)s %(message)s')

# os.system(f'bash {path}/longitude.sh')
thing=os.listdir(f'{path}/{day}/interpolation')
string=[]
string2=[]
for item in thing:
    if item.endswith(".csv"):
         string.append(item)
    if item.endswith(".csv.Y"):
         string2.append(item)
string=sorted(string)
string2=sorted(string2)

def plottinglatlon(string,string2,start,end,path):
    proj='-JM15C -P'
    lims1='-R0.7/9.6/49/58.3'
    lims2='-R3.3/9.6/49/54'
    for item in string[start:end]:
        with lock:
            item2=item.strip('.csv')
            nr=item2.strip('minute_')
            logging.info(f'Thread {thread} has obtained latlock for step {nr}.')
            time1=[int(int(nr)/60),int(nr)%60]
            if time1[0]<10:
                time1[0]=f'0{time1[0]}'
            if time1[1]<10:
                time1[1]=f'0{time1[1]}'
            os.system(f'gmt xyz2grd {path}/{day}/interpolation/{item} -G{path}/{day}/interpolation/gridlat{nr}.grd -I0.05 -V -N0 {lims1}')
            psfile1=f'{path}/{day}/interpolation/minlat_{nr}.ps'
            os.system(f'gmt pscoast {proj} {lims2} -W0.25p -Ggrey -Slightblue -N1/0.25p -Df -K> {psfile1}' )
            os.system(f'gmt psbasemap {proj} {lims2} -Ba1 -BWeSn+t"Bx at {day2} -- {time1[0]}:{time1[1]}" -O -K>> {psfile1}' )
            os.system(f'gmt grdcontour {path}/{day}/interpolation/gridlat{nr}.grd -C10 -A50+f20p {proj} {lims2} -O >>{psfile1}')
            logging.info(f'Thread {thread} has released latlock.')
        os.system(f'convert -density 300 {psfile1} {path}/{day}/interpolation/minlat_{nr}.png')
        os.system(f'rm {psfile1}')
        os.system(f'rm {path}/{day}/interpolation/gridlat{nr}.grd')
        logging.info(f'Thread {thread} has finished plotting lat for step {nr}.')  
              
    for item in string2[start:end]:
        with lock2:
            item2=item.strip('.csv.Y')
            nr=item2.strip('minute_')
            logging.info(f'Thread {thread} has obtained lonlock for step {nr}.')
            time2=[int(int(nr)/60),int(nr)%60]
            if time2[0]<10:
                time2[0]=f'0{time2[0]}'
            if time2[1]<10:
                time2[1]=f'0{time2[1]}'
            os.system(f'gmt xyz2grd {path}/{day}/interpolation/{item} -G{path}/{day}/interpolation/gridlon{nr}.grd -I0.05 -V -N0 {lims1}')
            psfile2=f'{path}/{day}/interpolation/minlon_{nr}.ps'
            os.system(f'gmt pscoast {proj} {lims2} -W0.25p -Ggrey -Slightblue -N1/0.25p -Df -K> {psfile2}' )
            os.system(f'gmt psbasemap {proj} {lims2} -Ba1 -BWeSn+t"By at {day2} -- {time2[0]}:{time2[1]}" -O -K>> {psfile2}' )
            os.system(f'gmt grdcontour {path}/{day}/interpolation/gridlon{nr}.grd -C10 -A50+f20p {proj} {lims2} -O >>{psfile2}')
            logging.info(f'Thread {thread} has released lonlock.')
        os.system(f'convert -density 300 {psfile2} {path}/{day}/interpolation/minlon_{nr}.png')
        os.system(f'rm {psfile2}')
        os.system(f'rm {path}/{day}/interpolation/gridlon{nr}.grd')
        logging.info(f'Thread {thread} has finished plotting lon for step {nr}.
# os.system(f'rm {path}/*.csv.Y')

n=6
nrsteps=int(1440/n)
threads=list()
for index in range(n):
    q=Process(target=plottinglatlon, args=(string, string2, nrsteps*index, nrsteps*(index+1), path))
    threads.append(q)
    q.start()
for thread in threads:
    thread.join()
logging.info('Plotting completed!')

In [52]:
# eenmalig nodig, nu niet meer
path='/usr/people/out/Documents/Magnetic_field/station_results/interpolation/'
for i in range(141,142):
    os.system(f'mv {path}minute={i}.csv {path}minute_{i}.csv')
################## Tijdelijk ################################
thing=os.listdir(f'{path}')
string1=[]
for item in thing:
    if item.endswith(".csv"):
        string1.append(item)
string1=sorted(string1)
print(string1)

for nr in range(141,len(string1)+141): 
    os.system(f'mv {path}/minute={nr}.csv {path}/minute_{nr}.csv')
os.system(f'bash {path}/latlon.sh')
#############################################################

In [3]:
import os
path='/usr/people/out/Documents/Magnetic_field/station_results/new_29-10-2003'
os.system(f'ffmpeg -framerate 20 -pattern_type glob -i "{path}/minlat_????.png" {path}/outputlat.mp4')
os.system(f'ffmpeg -framerate 20 -pattern_type glob -i "{path}/minlon_????.png" {path}/outputlon.mp4')

0

In [1]:
# plot larger area together with observations
def plotheadlen(string,nrstart,nrstop,stringval,location,path):
    proj='-JM15C -P'
    lims='-R3.3/9.5/49.5/54'
    localvar.counter=nrstart-1 #create counter for identifying correct line in station files
    scaling=400.
    for item in string[nrstart:nrstop]:
        localvar.counter+=1 
        localvar.newfile=pd.DataFrame(columns=['lon','lat','degree','BH'])
        for station in range(len(location)):
            localvar.statval=pd.read_csv(f'{stringval[station]}/allresults.csv', delimiter = ';')
            
            if localvar.statval.at[localvar.counter,"B_theta (nt)"]<0:
                localvar.correction=180
            else:
                localvar.correction=0
                
            localvar.newfile.loc[station]=location[station,1], location[station,0], np.degrees(np.arctan(localvar.statval.at[localvar.counter,"B_phi (nt)"]/localvar.statval.at[localvar.counter,"B_theta (nt)"]))+localvar.correction, np.sqrt(localvar.statval.at[localvar.counter,"B_phi (nt)"]**2+localvar.statval.at[localvar.counter,"B_theta (nt)"]**2)/scaling
            #create vector file for stations
        with lock:
            item2=item.strip('.csv')
            nr=item2.strip('headlen_')
            time1=[int(int(nr)/60),int(nr)%60]
            print(nr)
            psfile=f'{path}/interpolation/vector_{nr}.ps'
            localvar.newfile.to_csv(path_or_buf=f'{path}/obs{nr}.txt', sep=' ', index=False, header=False) #write station variable to file, so that gmt can read it
            if time1[0]<10:
                time1[0]=f'0{time1[0]}'
            if time1[1]<10:
                time1[1]=f'0{time1[1]}'
            os.system(f'gmt pscoast {proj} {lims} -W0.25p -Ggrey -Slightblue -Df -N1/0.25p -K> {psfile}' )
            os.system(f'gmt psbasemap {proj} {lims} -Ba1 -BWeSn+t"Magnetic field at {day} -- {time1[0]}:{time1[1]}" -O -K>> {psfile}' )
            os.system(f'gmt psxy {proj} {lims} {path}/interpolation/{item} -SV0.01i/0.02i/0.02i -Gblack -O -K>> {psfile}' )
            os.system(f'gmt psxy {proj} {lims} {path}/obs{nr}.txt -SV0.025i/0.05i/0.05i -Gred -O >> {psfile}' ) #vector: thickness line/thickness arrow/width arrow
        os.system(f'rm {path}/obs{nr}.txt')
        os.system(f'convert -density 300 {psfile} {path}/interpolation/headlen_{nr}.png')
        os.system(f'rm {psfile}')

import os
from multiprocessing import Process
from multiprocessing import Lock
import numpy as np
import pandas as pd
import threading
localvar=threading.local()
path='/nobackup/users/out/Magnetic_field/large_31-10-2003'
path2='/usr/people/out/Documents/Magnetic_field/'
day='31-10-2003'
lock=Lock()

thing=os.listdir(f'{path}/interpolation')
string=[]
os.system(f'ls {path}/interpolation/headlen*.csv > {path}/temp.txt')
f=open(f'{path}/temp.txt','r')
for item in f:
    item=item.strip('\n')
    item=item.strip(f'{path}/interpolation/')
    string.append(item)
string=sorted(string) #obtain locations of time dependant csv files, so that they can be found easier later
# print(string)
f.close()
os.system(f'rm {path}/temp.txt')

string2=[]
os.system(f'ls -d {path}/*_2003-10-31 > {path}/temp2.txt')
g=open(f'{path}/temp2.txt','r')
for item in g:
    item=item.strip('\n')
    string2.append(item)
string2=sorted(string2) #obtain location of station results
g.close()
os.system(f'rm {path}/temp2.txt')

string3=os.listdir(f'{path2}magnetic_data/{day}') #get location of stations
string3=sorted(string3) #sort alphabetically, otherwise problems later
location=np.zeros((len(string3),2))
for counter1,item in enumerate(string3):
    File=open(f'{path2}magnetic_data/{day}/{item}','r')
    for counter2,line in enumerate(File):
        if counter2==4:
            word=line.split()
            location[counter1,0]=word[2] #latitude
        if counter2==5:
            word=line.split()
            location[counter1,1]=word[2] #longitude
            break
    File.close()
        
n=6
nrsteps=int(8*60/n)
threads=list()
for index in range(n):
    q=Process(target=plotheadlen, args=(string,nrsteps*index,nrsteps*(index+1),string2,location,path))
    threads.append(q)
    q.start()
for thread in threads:
    thread.join()

headlen_0000.csv
headlen_0080.csv
headlen_0160.csv
headlen_0240.csv
headlen_0320.csv
headlen_0400.csv
0000
0160
0080
0240
headlen_0001.csv
0320
headlen_0161.csv
0400
headlen_0081.csv
0001
headlen_0241.csv
0161
headlen_0321.csv
0081
headlen_0401.csv
0241
headlen_0002.csv
0321
headlen_0162.csv
0401
0002
headlen_0082.csv
headlen_0242.csv
0162
headlen_0322.csv
0082
headlen_0402.csv
0242
headlen_0003.csv
0322
headlen_0163.csv
0402
headlen_0083.csv
0003
headlen_0243.csv
0163
headlen_0323.csv
0083
headlen_0403.csv
0243
headlen_0004.csv
0323
headlen_0164.csv
0403
headlen_0084.csv
0004
headlen_0244.csv
0164
headlen_0324.csv
0084
headlen_0404.csv
0244
headlen_0005.csv
0324
headlen_0165.csv
0404
headlen_0085.csv
0005
headlen_0245.csv
0165
headlen_0325.csv
0085
headlen_0405.csv
0245
0325
headlen_0006.csv
headlen_0166.csv
0405
headlen_0086.csv
0006
headlen_0246.csv
0166
headlen_0326.csv
0086
headlen_0406.csv
0246
headlen_0007.csv
0326
headlen_0167.csv
0406
headlen_0087.csv
0007
headlen_0247.csv
hea

headlen_0222.csv
0461
headlen_0142.csv
0062
headlen_0302.csv
0222
headlen_0382.csv
0142
headlen_0462.csv
0302
0382
headlen_0063.csv
headlen_0223.csv
0462
0063
headlen_0143.csv
headlen_0303.csv
0223
0143
headlen_0383.csv
headlen_0463.csv
0303
headlen_0064.csv
0383
headlen_0224.csv
0463
headlen_0144.csv
0064
headlen_0304.csv
0224
headlen_0384.csv
0144
headlen_0464.csv
0304
headlen_0065.csv
0384
headlen_0225.csv
0464
headlen_0145.csv
0065
headlen_0305.csv
0225
headlen_0385.csv
0145
headlen_0465.csv
0305
headlen_0066.csv
0385
headlen_0226.csv
0465
headlen_0146.csv
0066
headlen_0306.csv
0226
headlen_0386.csv
0146
headlen_0466.csv
0306
headlen_0067.csv
0386
headlen_0227.csv
0466
headlen_0147.csv
0067
headlen_0307.csv
0227
headlen_0387.csv
0147
headlen_0467.csv
0307
headlen_0068.csv
0387
headlen_0228.csv
0467
headlen_0148.csv
0068
headlen_0308.csv
0228
headlen_0388.csv
0148
headlen_0468.csv
0308
headlen_0069.csv
0388
headlen_0229.csv
0468
headlen_0149.csv
headlen_0309.csv
headlen_0389.csv
006

In [2]:
import os
path='/nobackup/users/out/Magnetic_field/large_31-10-2003'
os.system(f'ffmpeg -framerate 24 -pattern_type glob -i "{path}/interpolation/headlen_????.png" {path}/outputvector.mp4')

0

In [2]:
# plot electric fields over Netherlands
def plottingelectric(thread,string,start,end,path,day,direction):
    proj='-JM15C -P'
#     lims1='-R0.7/9.6/49/58.3' #for interpolation grid
    lims2='-R3.3/9.6/49/54' #for plotting
    for item in string[start:end]:
        with lock:
            item2=item.strip('.csv')
            nr=item2.strip('electric_')
            logging.info(f'Thread {thread} has obtained lock for step {nr} in {direction}-direction.')
            time1=[int(int(nr)/60),int(nr)%60]
            if time1[0]<10:
                time1[0]=f'0{time1[0]}'
            if time1[1]<10:
                time1[1]=f'0{time1[1]}'
            print(f'{path}/{item}')
            os.system(f'gmt xyz2grd {path}/{item} -G{path}/grid{nr}.grd -I0.05 -V -N0 {lims2}')
            psfile1=f'{path}/electric_{nr}.ps'
            os.system(f'gmt pscoast {proj} {lims2} -W0.25p -Ggrey -Slightblue -N1/0.25p -Df -K> {psfile1}' )
            os.system(f'gmt psbasemap {proj} {lims2} -Ba1 -BWeSn+t"E_{direction} at {day} -- {time1[0]}:{time1[1]}" -O -K>> {psfile1}' )
            os.system(f'gmt grdcontour {path}/grid{nr}.grd -C0.0000001 -A0.00000005+f20p {proj} {lims2} -O >>{psfile1}')
            logging.info(f'Thread {thread} has released lock.')
        os.system(f'convert -density 300 {psfile1} {path}/electric_{nr}.png')
        os.system(f'rm {psfile1}')
        os.system(f'rm {path}/grid{nr}.grd')
        logging.info(f'Thread {thread} has finished plotting the electric field for step {nr} in {direction}-direction.') 
    logging.info(f'Thread {thread} is completely finished!')

In [3]:
# plot electric fields over Netherlands
from multiprocessing import Process
import os
from multiprocessing import Lock
import logging

lock=Lock()

# magnetische contouren latitude
path='/nobackup/users/out/Magnetic_field'
day='Halloween' #real map location
day2='Halloween' #name for title plots

logging.basicConfig(filename=f'{path}/{day}/logbookplotelectric.log', level=logging.DEBUG, format='%(asctime)s %(message)s')

# we start with the northern electric field
path_to_fileN=f'{path}/{day}/electric_field'
files=os.listdir(f'{path_to_fileN}')
string=[]
for item in files:
    if item.endswith(".csv"):
         string.append(item)
string=sorted(string)

n=6
nrsteps=int(1440*3/n)
threads=list()
for index in range(n):
    q=Process(target=plottingelectric, args=(index+1,string, nrsteps*index, nrsteps*(index+1), path_to_fileN, day2, 'horizontal'))
    threads.append(q)
    q.start()
for thread in threads:
    thread.join()
logging.info('Plotting completed for Northern field!')

# # now we do it for the eastern field
# path_to_fileE=f'{path}/{day}/electric_field_east'
# files=os.listdir(f'{path_to_fileE}')
# string=[]
# for item in files:
#     if item.endswith(".csv"):
#          string.append(item)
# string=sorted(string)

# n=6
# nrsteps=int(1440/n)
# threads=list()
# for index in range(n):
#     q=Process(target=plottingelectric, args=(index+1,string, nrsteps*index, nrsteps*(index+1), path_to_fileE, day2, 'y'))
#     threads.append(q)
#     q.start()
# for thread in threads:
#     thread.join()
# logging.info('Plotting completed for Eastern field!')

/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0000.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0720.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1440.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2160.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2880.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3600.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0001.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0721.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2161.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1441.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2881.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3601.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/elec

/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1457.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2897.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0018.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0738.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2178.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3618.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1458.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2898.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0019.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0739.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2179.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3619.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/elec

/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1475.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0036.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0756.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2196.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3636.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2913.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1476.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0037.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0757.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2197.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3637.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2914.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/elec

/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3653.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0054.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1493.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0774.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2930.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2214.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3654.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0055.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1494.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0775.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2931.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2215.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/elec

/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1510.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0072.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2946.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0792.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3672.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2232.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1511.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0073.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0793.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2947.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3673.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2233.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/elec

...

/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1357.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_4243.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2777.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3516.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2079.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0644.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_4244.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1358.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2778.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3517.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2080.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0645.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/elec

/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0661.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_4261.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2794.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3534.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1375.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2097.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0662.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_4262.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2795.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3535.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1376.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2098.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/elec

/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2114.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1392.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0679.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_4279.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3552.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2812.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2115.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1393.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0680.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_4280.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3553.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2813.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/elec

/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3569.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2829.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2132.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0697.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_4297.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3570.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1410.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2830.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2133.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0698.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_4298.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3571.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/elec

/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_4314.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3587.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2847.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2150.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1427.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0715.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_4315.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_3588.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2848.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_2151.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_1428.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/electric_0716.csv
/nobackup/users/out/Magnetic_field/Halloween/electric_field/elec

In [14]:
def plot_electric_vec(string,nrstart,nrstop,path):
    proj='-JM15C -P'
    lims='-R3/10/48.7/54.3'
    for item in string[nrstart:nrstop]:
        with lock:
            item2=item.strip('.csv')
            nr=item2.strip('electric_')
            print(nr)
            time1=[int(int(nr)/60),int(nr)%60]
            psfile=f'{path}/electric_vector_{nr}.ps'
            if time1[0]<10:
                time1[0]=f'0{time1[0]}'
            if time1[1]<10:
                time1[1]=f'0{time1[1]}'
            os.system(f'gmt pscoast {proj} {lims} -W0.25p -Ggrey -Slightblue -Df -N1/0.25p -K> {psfile}' )
            os.system(f'gmt psbasemap {proj} {lims} -Ba1g1 -BWeSn+t"Electric field at {day} -- {time1[0]}:{time1[1]}" -O -K>> {psfile}' )
            os.system(f'gmt psxy {proj} {lims} {path}/{item} -SV0.001i/0.02i/0.02i -Gblack -O -K>> {psfile}' )
        os.system(f'convert -density 300 -background transparent {psfile} {path}/VecElec_{nr}.png')
        os.system(f'rm {psfile}')

In [16]:
# plot larger area together with observations
import os
from multiprocessing import Process
from multiprocessing import Lock
import numpy as np
import pandas as pd
import threading
localvar=threading.local()
path='/usr/people/out/Documents/Magnetic_field/station_results'
day='new_29-10-2003'
lock=Lock()

thing=os.listdir(f'{path}/{day}/electric_field_horizontal')
string=[]
os.system(f'ls {path}/{day}/electric_field_horizontal/electric_*.csv > {path}/{day}/temp.txt')
f=open(f'{path}/{day}/temp.txt','r')
for item in f:
    item=item.rstrip('\n')
    item=item.lstrip("/usr/people/out/Documents/Magnetic_field/station_results/new_29-10-2003/")
    item=item.lstrip('horizontal/')
    string.append(item)

string=sorted(string) #obtain locations of time dependant csv files, so that they can be found easier later

f.close()
os.system(f'rm {path}/{day}/temp.txt')
        
n=6
nrsteps=int(1440/6)
threads=list()
for index in range(n):
    q=Process(target=plot_electric_vec, args=(string,nrsteps*index,nrsteps*(index+1),f'{path}/{day}/electric_field_horizontal'))
    threads.append(q)
    q.start()
for thread in threads:
    thread.join()

0000
0240
0480
0720
0960
1200
0001
0241
0481
0721
0961
1201
0002
0242
0482
0722
0962
1202
0003
0243
0483
0723
0963
0004
1203
0244
0484
0724
0964
0005
0245
1204
0485
0725
0965
0006
0246
1205
0486
0726
0966
0007
0247
1206
0487
0727
0967
0008
0248
1207
0488
0728
0968
0009
0249
1208
0489
0729
0969
0010
0250
1209
0490
0730
0970
0011
0251
1210
0491
0731
0971
0012
0252
1211
0492
0732
0972
0013
0253
1212
0493
0733
0973
0014
0254
1213
0494
0734
0974
0015
0255
1214
0495
0735
0975
0016
0256
1215
0496
0736
0976
0257
0017
1216
0497
0737
0977
0258
0018
1217
0498
0738
0978
0259
0019
1218
0499
0739
0979
0020
0260
1219
0500
0740
0980
0021
0261
1220
0501
0741
0981
0022
0262
1221
0502
0742
0982
0023
0263
1222
0503
0743
0983
0024
0264
1223
0504
0744
0984
0025
0265
1224
0505
0745
0985
0026
0266
1225
0506
0746
0986
0027
0267
1226
0507
0747
0987
0028
0268
1227
0508
0748
0988
0029
0269
1228
0509
0749
0989
0030
0270
1229
0510
0750
0990
0031
0271
1230
0511
0751
0991
0032
0272
1231
0512
0752
0033
0992
0273
1232


In [4]:
import os
path='/nobackup/users/out/Magnetic_field/Halloween'
os.system(f'ffmpeg -framerate 24 -pattern_type glob -i "{path}/electric_field/electric_????.png" {path}/outputVecElec.mp4')

0